In [ ]:
import cv2
import os
import numpy as np
from sklearn.model_selection import train_test_split

def load_frames(directory , image_size=(112, 112)):
    images = []
    for root, dirs, files in os.walk(directory):
      for file in files:
        image_path = os.path.join(root, file)
        image = cv2.imread(image_path)

        if image is not None:
          image = cv2.resize(image, image_size)
          image = image.astype(np.float32) / 255.0  # Normalize to [0, 1]
          images.append(image)

    images = np.array(images)
    return images

In [ ]:
import numpy as np
import csv
import os

def file_text(video_path):
  for root, dirs, files in os.walk(video_path):
    for file in files:
      print(file)
      if file.endswith(".mp4"):
        video_path = os.path.join(root, file)
        
  # Open the MP4 file using OpenCV
  cap = cv2.VideoCapture(video_path)

  frames = []

  # Read frames until the video ends or the end time is reached
  while cap.isOpened():
    ret, frame = cap.read()

    if not ret :
        break

    frames.append(frame)
  print(len(frames))
  cap.release()

  path = '/kaggle/working/Frames' 
  if not os.path.exists(path):
    os.makedirs(path)

  for j, frame in enumerate(frames):
      final_frame = mouthDetection(frame)
      if np.all(final_frame == 0):
        print("not valid")
        continue
      cv2.imwrite(path+'/'+'%d.jpg' %j,final_frame)
      # res = mouthDetection(frame)
      # norm_res= normalize_frames(res)
      # word_frames.append(norm_res)

In [ ]:
import cv2
def mouthDetection(image):

  #from google.colab.patches import cv2_imshow

  # Load the pre-trained Haar Cascade classifier for face detection
  face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

  # Convert the image to grayscale
  gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

  # Perform face detection
  # minneighbors 16
  faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
  local_lip = lip_localization(gray)
  counter = 0
  mouth_gray = 0
  # Iterate over the detected faces
  for (x, y, w, h) in faces:
      # Define the region of interest (ROI) for the mouth
      roi_x = x
      roi_y = y + int(h * 2 / 3)  # Adjust the y-coordinate to focus on the lower part of the face
      roi_w = w
      roi_h = int(h / 3)

      # Extract the mouth region from the grayscale image
      mouth_gray = local_lip[roi_y:roi_y + roi_h, roi_x:roi_x + roi_w]


  return mouth_gray

In [ ]:
pip install dlib


In [ ]:
import numpy as np
import dlib

def lip_localization(image):
  # Load the detector
  detector = dlib.get_frontal_face_detector()
  # Load the predictor
  predictor = dlib.shape_predictor("/kaggle/input/face-landmarks/shape_predictor_68_face_landmarks.dat")
  # Convert image into grayscale
  #gray = cv2.cvtColor(src=image, code=cv2.COLOR_BGR2GRAY)
  # Use detector to find landmarks
  faces = detector(image)
  for face in faces:
      x1 = face.left() # left point
      y1 = face.top() # top point
      x2 = face.right() # right point
      y2 = face.bottom() # bottom point
      # Create landmark object
      landmarks = predictor(image=image, box=face)
      # Loop through all the points
      for n in range(48, 60):
          x = landmarks.part(n).x
          y = landmarks.part(n).y
          # Draw a circle
          cv2.circle(img=image, center=(x, y), radius=1, color=(255, 255, 255), thickness=-1)
  return image

In [ ]:
file_text("/kaggle/input/gbDataset")

In [ ]:
frames = load_frames("/kaggle/working/Frames")
frames = frames.reshape((-1, 1,112, 112, 3))


In [ ]:
import numpy as np
import tensorflow as tf
import keras
#from tensorflow_addons.layers import TFSMLayer
#tf.keras.models.load_weights("/content/model.h5")
#keras.layers.TFSMLayer("model1", call_endpoint="serving_default")
reconstructed_model =  tf.keras.models.load_model("/kaggle/input/final-ds/model (1).h5")

In [ ]:
y_predict = reconstructed_model.predict(frames)
y_categorical = np.argmax(y_predict, axis=1)

In [ ]:
import os
def read_words_to_dict(filename):
        with open(filename, 'r') as file:
            word_list = file.read().splitlines()
        word_dict = {}
        for line in word_list:
            index, word = line.split(':')
            word_dict[index] = word

        return word_dict

word_dict = read_words_to_dict('/kaggle/input/final-ds/class_names.txt')
print(word_dict.keys())

In [ ]:
fr = np.median(y_categorical)
directory_path = "/kaggle/working/"

# Create the directory if it doesn't exist
if not os.path.exists(directory_path):
    os.makedirs(directory_path)

# Construct the full file path
file_path = os.path.join(directory_path, "arabic_word.txt")

# Open the file in write mode with UTF-8 encoding
with open(file_path, "w", encoding="utf-8") as file:
    # Write the Arabic word to the file
    file.write(word_dict[str(int(fr))])